In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [2]:
import pandas as pd 
import numpy as np 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/Matrix_repo"

/content/drive/My Drive/Colab Notebooks/Matrix_repo


In [4]:
ls data/

shoes_prices2.csv  shoes_prices.csv


In [0]:
df=pd.read_csv('data/shoes_prices.csv', low_memory=False)

In [0]:
df_usd = df[df.prices_currency == 'USD'].copy()
df_usd["prices_amountmin"] = df_usd.prices_amountmin.astype(np.float)
filter_max = np.percentile(df_usd["prices_amountmin"], 99)
df_usd_filter =df_usd[df_usd["prices_amountmin"] < filter_max]

df=df_usd_filter

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X=df[feats].values
  y=df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring= 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [8]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(-58.38655694633361, 4.223555478221712)

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

Feature engineering


In [0]:
df.head()

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) =='nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item ['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key]=value

  return output_dict  

df['features_parsed'] = df['features'].map(parse_features)

In [0]:
df['features_parsed'].head().values

In [0]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)

In [0]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
df.columns

In [0]:
keys_stat={}
for key in keys:
 keys_stat[key] = df[False== df[get_name_feat(key)].isnull()].shape[0] / df.shape[0]*100

In [0]:
{k:v for k,v in keys_stat.items() if v> 30}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_color_cat'] = df['feat_color'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_material_cat'] = df['feat_material'].map(lambda x: str(x).lower()).factorize()[0]


In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand ==df.feat_brand].shape

In [0]:
feats = ['']

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat' ]

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

In [0]:
X = df [feats].values
y = df ['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5,n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance (m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names = feats)

In [0]:
df['brand'].value_counts(normalize=True)

In [0]:
df[df['brand'] == 'nike'].features_parsed.head().values

In [0]:
!git add 'D5.ipynb'

In [0]:
!git config --global user.email "Justynaha@gmail.com"
!git config --global user.name "Justyna"

In [0]:
!git commit -m 'More modeling on Mens Shoe Prices dataset from DataWorld'

In [0]:
!git push -u origin master